In [1]:
import rospy
from giskardpy.python_interface.python_interface import GiskardWrapper
import numpy as np
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped
from tf.transformations import quaternion_from_matrix, quaternion_about_axis
from giskardpy.goals.adaptive_goals import CloseGripper, PouringAdaptiveTilt

In [2]:
rospy.init_node('giskard_notebook')

In [5]:
gs = GiskardWrapper()

In [6]:
# first start related scripts for BB detection and scene action reasoning
gs.motion_goals.add_motion_goal(motion_goal_class=CloseGripper.__name__,
                                name='openGripper',
                                as_open=True,
                                velocity_threshold=100,
                                effort_threshold=1,
                                effort=100)
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, 1, 0],
                                                                 [0, -1, 0, 0],
                                                                 [1, 0, 0, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 1.95
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.3

gs.motion_goals.add_cartesian_pose(goal_pose, 'hand_palm_link', 'map')
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

gs.motion_goals.add_motion_goal(motion_goal_class=CloseGripper.__name__,
                                name='closeGripper', effort=-220)
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

cup_pose = PoseStamped()
cup_pose.header.frame_id = 'free_cup'
cup_pose.pose.position = Point(0, 0, 0)
cup_pose.pose.orientation.w = 1

# add a new object at the pose of the pot and attach it to the right tip
gs.world.add_box('cup1', (0.07, 0.07, 0.28), pose=cup_pose, parent_link='hand_palm_link')
cup_pose.header.frame_id = 'free_cup2'
gs.world.add_box('cup2', (0.07, 0.07, 0.18), pose=cup_pose, parent_link='map')

# goal_pose.pose.position.x = 1.85
# goal_pose.pose.position.y = -0.7
# goal_pose.pose.position.z = 0.54
# goal_pose.pose.position.x = 1.75
# goal_pose.pose.position.y = -0.4
# goal_pose.pose.position.z = 0.6
goal_pose.header.frame_id = 'cup2'
goal_pose.pose.position = Point(-0.3, 0.2, 0.3)
# goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, 1, 0],
#                                                                  [0, -1, 0, 0],
#                                                                  [1, 0, 0, 0],
#                                                                  [0, 0, 0, 1]]))
tilt_axis = Vector3Stamped()
tilt_axis.header.frame_id = 'hand_palm_link'
tilt_axis.vector.z = 1
gs.motion_goals.add_motion_goal(motion_goal_class='PouringAdaptiveTilt',
                                name='pouring',
                                tip='hand_palm_link',
                                root='map',
                                tilt_angle=0.3,
                                pouring_pose=goal_pose,
                                tilt_axis=tilt_axis,
                                pre_tilt=False)
gs.motion_goals.allow_all_collisions()
gs.motion_goals.avoid_collision(0.01, 'cup1', 'cup2')
gs.add_default_end_motion_conditions()
gs.execute()

goal_pose.header.frame_id = 'map'
goal_pose.pose.position.x = 1.93
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.3

gs.motion_goals.add_cartesian_pose(goal_pose, 'hand_palm_link', 'map')
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

gs.motion_goals.add_motion_goal(motion_goal_class=CloseGripper.__name__,
                                name='openGripper',
                                as_open=True,
                                velocity_threshold=100,
                                effort_threshold=1,
                                effort=100)
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

goal_pose.pose.position.x = 1.4
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.4

gs.motion_goals.add_cartesian_pose(goal_pose, 'hand_palm_link', 'map')
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

KeyboardInterrupt: 